# Contents Covered
- Read mltable in a job
- Register an mltable as a data asset in Azure Machine Learning
- Consume registered mltable assets in a job

In [ ]:
# Pre-requisites

!pip uninstall azure-ai-ml
!pip install --pre azure-ai-ml
!pip uninstall mltable
!pip install --pre mltable
!pip install pandas

## Connect to Azure Machine Learning Workspace

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## MLTable definition file from local data path

In [ ]:
!cat ./sample_data/MLTable

In [ ]:
import mltable

tbl = mltable.load("./sample_data")
df = tbl.to_pandas_dataframe()
df.head(5)

## MLTable definition file from datastore uri path

1. get datastore uri path from registered data asset with uri_file type
2. construct mltable yaml with the datastore uri path and load it to pandas dataframe

- Get datastore uri path from registered data asset with uri_file type

In [ ]:
# get datastore uri from local data path
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_parquet_data = Data(
    path="./sample_data/data.parquet",
    type=AssetTypes.URI_FILE,
    description="Parquet data",
    name="v2_parquet_urifile",
)

my_parquet_data = ml_client.data.create_or_update(my_parquet_data)
print(my_parquet_data.path)

-Construct MLTable definition file from the datastore uri path(taking parquet file data source as example) and load it into pandas

In [ ]:
# helper function to create MLTable datastore uri path for parquet file
# uri_path is the datastore uri path in the format of long form datastore uri format: azureml://subscriptions/<sub-id>/resourcegroups/<resource-group>/workspaces/<workspace>/datastores/{datastore_name}/paths/{relative_data_path}
# mltable_folder is where to save the MLTable yaml
def mltable_from_parquet(uri_path, mltable_folder):
    import yaml
    import mltable

    # MLTable yaml dictionary
    mltable_from_parquet = {
        "paths": [{"file": f"{uri_path}"}],
        "transformations": ["read_parquet"],
    }

    temp_dir = tempfile.gettempdir()
    with open(f"{mltable_folder}/MLTable", "w") as mltable_yaml:
        yaml.dump(mltable_from_parquet, mltable_yaml, default_flow_style=False)

    return mltable.load(mltable_folder)

In [ ]:
import tempfile

temp_dir = tempfile.gettempdir()

# get datastore uri path from the registered asset
# save MLTable file to temp folder
mlt = mltable_from_parquet(my_parquet_data.path, temp_dir)
mlt.to_pandas_dataframe()

## Reading mltable in a job

In [ ]:
# read_mltable.py
import argparse
import mltable
import pandas

parser = argparse.ArgumentParser()
parser.add_argument("--input_data", type=str)
args = parser.parse_args()

tbl = mltable.load(args.input_data)
df = tbl.to_pandas_dataframe()
print(df.head(10))

In [ ]:
from azure.ai.ml import Input, command
from azure.ai.ml.entities import Data, Environment
from azure.ai.ml.constants import AssetTypes

env = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="environments/mltable_environment.yaml",
)

my_job_inputs = {"input_data": Input(type=AssetTypes.MLTABLE, path="./sample_data")}

job = command(
    code="./src",  # local path where the code is stored
    command="python read_mltable.py --input_data ${{inputs.input_data}}",
    inputs=my_job_inputs,
    environment=env,
    compute="cpu-cluster",
)

# submit the command job
returned_job = ml_client.jobs.create_or_update(job)
# get a URL for the status of the job
returned_job.studio_url

## Registering an mltable as an asset in Azure Machine Learning

- You can register an mltable as a data asset in Azure Machine Learning. The benefits of registering data are:

  - Easy to share with other members of the team (no need to remember file locations)
  - Versioning of the metadata (location, description, etc)

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_data = Data(
    path="./sample_data",
    type=AssetTypes.MLTABLE,
    description="Titanic Data",
    name="titanic-mltable-example",
)

my_mltable = ml_client.data.create_or_update(my_data)
mltable_version = my_mltable.version
print(mltable_version)

## Consume data assets in an Azure Machine Learning Job

In [ ]:
from azure.ai.ml import Input, command
from azure.ai.ml.entities import Data, Environment
from azure.ai.ml.constants import AssetTypes

env = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="environments/mltable_environment.yaml",
)

registered_data_asset = ml_client.data.get(
    name="titanic-mltable-example", version=mltable_version
)

my_job_inputs = {"input_data": Input(path=registered_data_asset.id)}

job = command(
    code="./src",
    command="python read_mltable.py --input_data ${{inputs.input_data}}",
    inputs=my_job_inputs,
    environment=env,
    compute="cpu-cluster",
)

# submit the command job
returned_job = ml_client.create_or_update(job)
# get a URL for the status of the job
returned_job.studio_url